참고 링크
- https://koreanfoodie.me/157
- https://nbviewer.jupyter.org/github/SDRLurker/deep-learning/blob/master/5%E1%84%8C%E1%85%A1%E1%86%BC.ipynb
- 밑바닥부터 시작하는 딥러닝1 책

## 5.7 오차 역전파법 구현하기

#### 5.7.1 신경망 학습의 전체 그림
##### 전제
__학습__ : 가중치와 편향을 훈련 데이터에 적응하도록 조정하는 과정 <br>

##### 1단계 - 미니배치
__미니배치 :__ 훈련 데이터 중 일부를 무작위로 가져옴 <br>
__목표 :__ 미니배치의 손실 함수 값을 줄이기
<br><br>

##### 2단계 - 기울기 산출
가중치 매개변수의 기울기를 구함 => 기울기는 __손실함수의 값을 가장 작게하는 방향을 제시__
<br><br>

##### 3단계 - 매개변수 갱신
가중치 매개변수를 기울기 방향으로 아주 조금 갱신
<br><br>

##### 4단계 - 반복
- 1~3단계를 반복<br>
- <u>오차역전법이 등장하는 단계는 두번째인 __기울기 산출__</u>
- <u>느린 수치 미분과 달리, __기울기를 효율적이고 빠르게__ 구할 수 있음</u>

##### 5.7.2 오차역전파법을 적용한 신경망 구현하기

계층을 사용함으로써 __predict()__와 기울기를 구하는 __gradient()__ 계층의 전파만으로 동작이 이뤄진다.

![](img/mnist-affine.png)
__위와 같은 Layer를 갖는 Neural Net구현하기__

In [2]:
# https://github.com/WegraLee/deep-learning-from-scratch/blob/master/ch05/two_layer_net.py 참고
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict



class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
        self.params['b2'] = np.zeros(output_size)

        # 계층 생성 
        self.layers = OrderedDict()                                           ###
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1']) ###
        self.layers['Relu1'] = Relu()                                         ###
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2']) ###

        self.lastLayer = SoftmaxWithLoss()                                    ###
        
    def predict(self, x):
        for layer in self.layers.values():                                    ###
            x = layer.forward(x)                                              ###
        
        return x
        
    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
    def gradient(self, x, t):
        # forward
        self.loss(x, t)                      ###

        # backward
        dout = 1                             ###
        dout = self.lastLayer.backward(dout) ###
        
        layers = list(self.layers.values())  ###
        layers.reverse()                     ###
        for layer in layers:                 ###
            dout = layer.backward(dout)      ###

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads

> __\### 으로 중요코드 표시__
> - 순전파 때는 추가한 순서대로 각 계층의 forward() 메서드를 호출
> - 역전파 때는 계층을 반대 순서로 호출
> - 신경망의 구성 요소를 '계층'으로 구현한 덕분에 신경망을 쉽게 구축
>     - __=>__ 레고 블록을 조립하듯 필요한 만큼 계층을 더 추가하면 됨

##### 5.7.3 오차역전파법으로 구한 기울기 검증하기
- 수치미분은 매우 느리다는것을 보여주기 위해 구현

In [4]:
from dataset.mnist import load_mnist
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)
x_batch = x_train[:3]
t_batch = t_train[:3]

In [5]:
%timeit network.numerical_gradient(x_batch, t_batch)

9.56 s ± 893 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%timeit network.gradient(x_batch, t_batch)

277 µs ± 29.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


> - 수치미분(numerical_gradient) 속도: 9.56초
> - 오차역전법(gradient) 속도: 277 µs, 0.000277초
> - 약 34,000배 속도차이가 남
> - __수치 미분을 구현하긴 쉽지만, 속도가 오차역전법(gradient)에 비해 매우 느림__

##### 5.7.4 오차역전파법을 사용한 학습 구현하기

In [8]:
import numpy as np
from dataset.mnist import load_mnist
#from two_layer_net import TwoLayerNet

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산
    #grad = network.numerical_gradient(x_batch, t_batch) # 수치 미분 방식
    grad = network.gradient(x_batch, t_batch) # 오차역전파법 방식(훨씬 빠르다)
    
    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

0.09948333333333333 0.1024
0.9026 0.9056
0.9217333333333333 0.9243
0.93285 0.9336
0.9426 0.9402
0.9510333333333333 0.9486
0.9542833333333334 0.9518
0.9607833333333333 0.9552
0.9645333333333334 0.9576
0.9671 0.9605
0.9708833333333333 0.9655
0.9727 0.9648
0.97505 0.9665
0.9760333333333333 0.9677
0.9763833333333334 0.9676
0.9788833333333333 0.967
0.9787833333333333 0.9704


## 5.8 정리
- 계산 그래프를 이용하여 신경망의 동작과 오차역전파법을 설명
- 모든 계층에서 forward와 backward 메서드를 구현
    - => __가중치 매개변수의 기울기를 효율적으로 구할 수 있음__
    
##### 이번 장에서 배운 것
- 계산 그래프를 이용하면 계산 과정을 시각적으로 파악 가능
- __오차역전파법 : 신경망의 구성 요소를 계층으로 구현하여 기울기를 효율적으로 계산__